# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION =
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')

except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


## Query 1 - Create the sessions table

In [8]:
sessions_table_creation = "create table if not exists sessions "
sessions_table_creation = sessions_table_creation + "(session_Id int, item_In_Session int, artist text, song_title text, song_length float, PRIMARY KEY (session_Id, item_In_Session))"
session.execute(sessions_table_creation)

#### Insert data into Sessions table from CSV file

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        sessions_data_insert = "INSERT INTO sessions (session_Id, item_In_Session, artist, song_title, song_length)"
        sessions_data_insert = sessions_data_insert + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(sessions_data_insert, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Validate the data in the Sessions table
Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [10]:
query_1 = "SELECT artist, song_title, song_length FROM sessions WHERE session_Id = 338 AND item_In_Session = 4"
rows = session.execute(query_1)
for row in rows:
    print(row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## Query 2 - Create the Users Table

In [11]:
users_table_creation = "create table if not exists users "
users_table_creation = users_table_creation + "(artist_name text, song_title text, item_in_session int, first_name text, last_name text, user_id int, session_id int, PRIMARY KEY (user_id, session_Id, item_in_session))"
session.execute(users_table_creation)

#### Insert data into Users table from CSV file

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        users_data_insert = "INSERT INTO users (artist_name, song_title, item_in_session, first_name, last_name, user_id, session_id)"
        users_data_insert = users_data_insert + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(users_data_insert, (line[0], line[9], int(line[3]), line[1], line[4], int(line[10]), int(line[8])))

#### Validate the data in the Artist table 
Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [13]:
query_2 = "SELECT artist_name, song_title, first_name, last_name FROM users WHERE user_id = 10 AND session_id = 182"
rows = session.execute(query_2)
for row in rows:
    print(row.artist_name, row.song_title, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Query 3 - Create the Songs table

In [14]:
songs_table_creation = "create table if not exists songs "
songs_table_creation = songs_table_creation + "(first_name text, last_name text, song_title text, user_id int, PRIMARY KEY (song_title, user_id))"
session.execute(songs_table_creation)

#### Insert data into Songs table from CSV file

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        songs_data_insert = "INSERT INTO songs (first_name, last_name, song_title, user_id)"
        songs_data_insert = songs_data_insert + "VALUES (%s, %s, %s, %s)"
        session.execute(songs_data_insert, (line[1], line[4], line[9], int(line[10])))

#### Validate the data in the Songs table
Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [16]:
query_3 = "SELECT first_name, last_name FROM songs WHERE song_title = 'All Hands Against His Own'"
rows = session.execute(query_3)
for row in rows:
    print(row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
drop_sessions = "drop table if exists sessions"
session.execute(drop_sessions)

drop_users = "drop table if exists users"
session.execute(drop_users)

drop_songs = "drop table if exists songs"
session.execute(drop_songs)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()